In [21]:
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.llms.ollama import Ollama
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [3]:
loader = PyPDFLoader("../data/photoshop_reference.pdf")

In [4]:
docs = loader.load()

In [5]:
len(docs)

1017

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

docs = text_splitter.split_documents(docs)

In [9]:
len(docs)

2683

In [13]:
model = "Alibaba-NLP/gte-large-en-v1.5"
model_kwargs = {'device':'cpu', 'trust_remote_code': True}
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=model,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/Users/kumarshubham/personal/photoshop-rag/.venv/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/kumarshubham/personal/photoshop-rag/.venv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [16]:
db = FAISS.from_documents(docs, embeddings)

In [24]:
template = """Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Give a detailed answer in upto 5 sentences.
    {context}
    Question: {question}
    Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

In [25]:
llm = Ollama(model="llama3:8b", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=db.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

In [28]:
result = qa_chain({"query": "What are the system requirements for Adobe Photoshop and can I run it with 1 GB of RAM?"})

Based on the provided context, here is a detailed answer to your question:
To determine whether you can run Adobe Photoshop with 1 GB of RAM, we need to look at the minimum system requirements mentioned in the context. According to the text, the recommended RAM for Photoshop is 8 GB, but the minimum requirement is 2 GB or more of RAM. Since you have 1 GB of RAM, which is less than the minimum requirement, it's unlikely that you can run Adobe Photoshop with this amount of RAM. However, if you want to try running Photoshop with 1 GB of RAM, it might be slow and unstable due to limited memory availability.